In [1]:
import pandas as pd
import numpy as np

In [2]:

github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/c445434454be50934cd75c661626ae05c3ec8126/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [3]:
df = df.iloc[:300]

In [4]:
model_name = 'multi-qa-mpnet-base-dot-v1'
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(model_name)


/home/julie/miniconda3/envs/exp-tracking-env/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
df.iloc[0]

answer_llm     You can sign up for the course by visiting the...
answer_orig    Machine Learning Zoomcamp FAQ\nThe purpose of ...
document                                                0227b872
question                     Where can I sign up for the course?
course                                 machine-learning-zoomcamp
Name: 0, dtype: object

In [6]:
answer_llm = df.iloc[0].answer_llm
v = model.encode(answer_llm)
v[0], v.dot(v), len(v)

(-0.42244676, 34.820374, 768)

In [7]:
type(df.iloc)

pandas.core.indexing._iLocIndexer

In [9]:
evaluations = []
ans = {}
    #print(doc)
ans1 = df.answer_llm.to_list()
ans2 = df.answer_orig.to_list()
for a,b in zip(ans1, ans2):
    a_llm = model.encode(a)
    a_orig = model.encode(b) 
    evaluations.append(a_llm.dot(a_orig))
np.percentile(evaluations, 75)

31.674306392669678

In [10]:
evaluations1 = []
for a,b in zip(ans1, ans2):
    a_llm = model.encode(a)
    a_llm = a_llm/np.sqrt(a_llm.dot(a_llm))
    
    a_orig = model.encode(b) 
    a_orig = a_orig/np.sqrt(a_orig.dot(a_orig))

    evaluations1.append(a_llm.dot(a_orig))
np.percentile(evaluations1, 75)

0.8362347036600113

In [11]:
df.iloc[10]

answer_llm     Yes, all sessions are recorded, so if you miss...
answer_orig    Everything is recorded, so you won’t miss anyt...
document                                                5170565b
question                    Are sessions recorded if I miss one?
course                                 machine-learning-zoomcamp
Name: 10, dtype: object

In [12]:
r = df.iloc[10]
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [13]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [14]:
(scores['rouge-1']['f'] + scores['rouge-2']['f'] + scores['rouge-l']['f'])/3 

0.35490034990035496

In [16]:
ddict = {}
ddict['rouge_1'] = []
ddict['rouge_2'] = []
ddict['rouge_l'] = []
ddict['rouge_avg'] = []

for j in range(len(df.index)):
    d = df.iloc[j]
    scores = rouge_scorer.get_scores(d['answer_llm'], d['answer_orig'])[0]
    ddict['rouge_1'].append(scores['rouge-1']['f'])
    ddict['rouge_2'].append(scores['rouge-2']['f'])
    ddict['rouge_l'].append(scores['rouge-l']['f'])
    ddict['rouge_avg'].append((scores['rouge-1']['f'] + scores['rouge-2']['f'] + scores['rouge-l']['f'])/3 )
    
df1 = pd.DataFrame(data=ddict)
#df1 = df1.replace(0, np.NaN)
df1.mean()['rouge_2']


0.20696501983423318